ESC324 
Classifying Movie Genres


# Data gathering

The code that is commented below is simply showing how we obtained data from Kaggle, but as we have added the data csvs to our repository, this does not need to be run. 

Set up to extract data from Kaggle is from the code in this website: https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/ 

In [ ]:
# set up kaggle to get data 
# only need to run once and use your personal kaggle api key (need to load kaggle.json to files)
# ! pip install -q kaggle
# ! mkdir ~/.kaggle
# ! cp /kaggle.json ~/.kaggle/ # contains personal kaggle api key
# ! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# Movies dataset
# ! kaggle datasets download -d rounakbanik/the-movies-dataset -f movies_metadata.csv
# # Netflix dataset
# ! kaggle datasets download -d satpreetmakhija/netflix-movies-and-tv-shows-2021
# # IMDB Movies dataset
# ! kaggle datasets download -d harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows

 49% 6.00M/12.2M [00:00<00:00, 54.5MB/s]
100% 12.2M/12.2M [00:00<00:00, 53.2MB/s]
  0% 0.00/1.07M [00:00<?, ?B/s]
100% 1.07M/1.07M [00:00<00:00, 16.0MB/s]
  0% 0.00/175k [00:00<?, ?B/s]
100% 175k/175k [00:00<00:00, 18.4MB/s]


Below code is used to get data for training and testing our models. 

In [ ]:
# unzip files 
! unzip movies_metadata.csv.zip
! unzip netflix-movies-and-tv-shows-2021.zip
! unzip imdb-dataset-of-top-1000-movies-and-tv-shows.zip

Archive:  movies_metadata.csv.zip
  inflating: movies_metadata.csv     
Archive:  netflix-movies-and-tv-shows-2021.zip
  inflating: netflixData.csv         
Archive:  imdb-dataset-of-top-1000-movies-and-tv-shows.zip
  inflating: imdb_top_1000.csv       


Extract relevant data from csvs


In [ ]:
import pandas as pd 
import json

def extract_genre(x):
  """
  Helper function for extracting genre from the Kaggle Movies dataset
  """
  for i in range(len(x)):
    x[i] = x[i]['name']
  return x

# Movies dataset (movies_metadata.csv)
# we want features: 'genres' (label), 'overview' (input)
df_1 = pd.read_csv('movies_metadata.csv')
# print(df_1.columns)
input_1 = df_1['overview']
output_1 = df_1['genres']

# data in kaggle dataset is in the form: '[{'id': 12, 'name': 'Adventure'},..'
# we want to extract the name of genres to a list of the form: ['Adventure',..]
output_1 = output_1.apply(lambda x: json.loads(x.replace("'", '"'))) # make the string into a list of dictionaries
output_1 = output_1.apply(extract_genre) # get the genre from the dictionaries

# print(len(input_1.index)) # size of dataset
# print(output_1[1]) # example genre output

# Netflix dataset (netflixData.csv)
# features: 'genres' (label), 'description' (input)
df_2 = pd.read_csv('netflixData.csv')
# print(df_2.columns)
input_2 = df_2['Description']
output_2 = df_2['Genres']

output_2 = output_2.str.split(', ') # make genres a list
# print(len(input_2.index)) # size of dataset
# print(output_2[2]) # example genre output

# IMDB dataset (imdb_top_1000.csv)
# features: 'Genre' (label), 'Overview' (input)
df_3 = pd.read_csv('imdb_top_1000.csv')
# print(df_3.columns)
input_3 = df_3['Overview']
output_3 = df_3['Genre']

output_3 = output_3.str.split(', ') # make genres a list
# print(len(input_3.index)) # size of dataset
# print(output_3[1]) # example genre output

# Human classification dataset (human_classification_training.csv)
df_4 = pd.read_csv('human_classification_training.csv')
# print(df_4.columns)
input_4 = df_4['Synopsis']
output_4 = df_4['Genre']

# print(len(input_3.index)) # size of dataset
# print(output_3[1]) # example genre output

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


45466
['Adventure', 'Fantasy', 'Family']
5967
['Documentaries', 'International Movies']
1000
['Crime', 'Drama']


Split into training, validation, and test data

In [ ]:
# combine all inputs together
inputs = pd.concat([input_1,input_2,input_3,inputs_4])
outputs = pd.concat([output_1, output_2,output_3,outputs_4])
# print(outputs[5]) # example of what the output genre data looks like

5                   [Action, Crime, Drama, Thriller]
5    [Dramas, International Movies, Romantic Movies]
5                         [Action, Adventure, Drama]
dtype: object


In [ ]:
# get train validate test split of data
from sklearn.model_selection import train_test_split

# 80% train : 10% validation : 10% test
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs,
    test_size=0.2, shuffle = True, random_state = 8)

# get validation split from test dataset
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, 
    test_size=0.5, random_state= 8) 


1    An organized crime dynasty's aging patriarch t...
1    When siblings Judy and Peter discover an encha...
1    As a grisly virus rampages a city, a lone man ...
dtype: object
1                                      [Crime, Drama]
1                        [Adventure, Fantasy, Family]
1    [Horror Movies, International Movies, Thrillers]
dtype: object
5243
5244
